In [123]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import math, time
import itertools
from sklearn import preprocessing
import datetime
from operator import itemgetter
from sklearn.metrics import mean_squared_error
from math import sqrt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import LSTM

* Get the stock data from Yahoo finance as a CSV for any company. 

* Drop all other columns (Volume, Year, Month and Day) and retain Open, High and Close - since I'm just predicting the close value for a day

In [124]:
df = pd.read_csv('Stock_Markets.csv')
df.head()

,Date,Open,High,Low,Close,Adj_Close,Volume,Stock
0,2/21/2023,204.990005,209.710007,197.220001,197.369995,197.369995,180018600,TSLA
1,2/22/2023,197.929993,201.990005,191.779999,200.860001,200.860001,191828500,TSLA
2,2/23/2023,203.910004,205.139999,196.330002,202.070007,202.070007,146360000,TSLA
3,2/24/2023,196.330002,197.669998,192.800003,196.880005,196.880005,142228100,TSLA
4,2/27/2023,202.029999,209.419998,201.259995,207.630005,207.630005,161028300,TSLA


In [125]:
def get_stock_data(stock, normalized=0):
    df = pd.read_csv('Stock_Markets.csv')

    df["Volume"] = df["Volume"] / 10000
    # df.drop(df.columns[[0,3,5,6, 7,8,9]], axis=1) 
    return df

In [126]:
# stock = input("Enter stock name: ")
stock = 'TSLA'

df = get_stock_data(stock,0)

# Checking if any data was found for the specified stock
if not df.empty:
    print("Data Retrieved")
else:
    print(f"No Data Found For {stock}.")

df.head()


Data Retrieved


,Date,Open,High,Low,Close,Adj_Close,Volume,Stock
0,2/21/2023,204.990005,209.710007,197.220001,197.369995,197.369995,18001.86,TSLA
1,2/22/2023,197.929993,201.990005,191.779999,200.860001,200.860001,19182.85,TSLA
2,2/23/2023,203.910004,205.139999,196.330002,202.070007,202.070007,14636.00,TSLA
3,2/24/2023,196.330002,197.669998,192.800003,196.880005,196.880005,14222.81,TSLA
4,2/27/2023,202.029999,209.419998,201.259995,207.630005,207.630005,16102.83,TSLA


In [127]:
df = df.drop(columns=['Low', 'Adj_Close', 'Volume', 'Stock', 'Date'])

In [128]:
# # Convert the `Date` feature to a datetime object.
# df['Date'] = pd.to_datetime(df['Date'])

# # Set the `Date` feature as the index of the DataFrame.
# df = df.set_index('Date')

* Let me save the data to a CSV file so that I can refer later (if needed :) ) 

In [129]:
today = datetime.date.today()
filename = stock+'_stock_%s.csv' % today
df.to_csv(filename)

Normalizing

In [130]:
df['Open'] = df['Open']/100
df['High'] = df['High']/100
df['Close'] = df['Close']/100
df.head()

,Open,High,Close
0,2.0499,2.0971,1.9737
1,1.9793,2.0199,2.0086
2,2.0391,2.0514,2.0207
3,1.9633,1.9767,1.9688
4,2.0203,2.0942,2.0763


In [131]:
#Load the data
def load_data(stock, seq_len):
    amount_of_features = len(stock.columns)
    data = stock.to_numpy() #pd.DataFrame(stock)
    sequence_length = seq_len + 1
    result = []
    for index in range(len(data) - sequence_length):
        result.append(data[index: index + sequence_length])

    result = np.array(result)
    row = round(0.9 * result.shape[0])
    train = result[:int(row), :]
    x_train = train[:, :-1]
    y_train = train[:, -1][:,-1]
    x_test = result[int(row):, :-1]
    y_test = result[int(row):, -1][:,-1]

    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], amount_of_features))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], amount_of_features))  

    return [x_train, y_train, x_test, y_test]

In [132]:
#Build the model
def build_model(layers):
    model = Sequential()

    model.add(LSTM(
        input_dim=layers[0],
        output_dim=layers[1],
        return_sequences=True))
    model.add(Dropout(0.2))

    model.add(LSTM(
        layers[2],
        return_sequences=False))
    model.add(Dropout(0.2))

    model.add(Dense(
        output_dim=layers[2]))
    model.add(Activation("linear"))

    start = time.time()
    model.compile(loss="mse", optimizer="rmsprop",metrics=['accuracy'])
    print("Compilation Time : ", time.time() - start)
    return model

def build_model2(layers):
        d = 0.2
        model = Sequential()
        model.add(LSTM(128, input_shape=(layers[1], layers[0]), return_sequences=True))
        model.add(Dropout(d))
        model.add(LSTM(64, input_shape=(layers[1], layers[0]), return_sequences=False))
        model.add(Dropout(d))
        model.add(Dense(16,kernel_initializer='uniform',activation='relu'))        
        model.add(Dense(1,kernel_initializer='uniform',activation='linear'))
        model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
        return model
def build_model2(layers):
        d = 0.2
        model = Sequential()
        model.add(LSTM(128, input_shape=(layers[1], layers[0]), return_sequences=True))
        model.add(Dropout(d))
        model.add(LSTM(64, input_shape=(layers[1], layers[0]), return_sequences=False))
        model.add(Dropout(d))
        model.add(Dense(16,kernel_initializer='uniform',activation='relu'))        
        model.add(Dense(1,kernel_initializer='uniform',activation='linear'))
        model.compile(loss='mse',optimizer='adam',metrics=['accuracy'])
        return model

Divide as Training and Test set

In [133]:
window = 22
X_train, y_train, X_test, y_test = load_data(df[::-1], window)
print("X_train", X_train.shape)
print("y_train", y_train.shape)
print("X_test", X_test.shape)
print("y_test", y_test.shape)

X_train (2210, 22, 3)
y_train (2210,)
X_test (246, 22, 3)
y_test (246,)


In [134]:
# model = build_model([3,lag,1])
model = build_model2([3,window,1])

Execute the trained model

In [135]:
model.fit(
    X_train,
    y_train,
    batch_size=512,
    nb_epoch=500,
    validation_split=0.1,
    verbose=1)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "C:\Users\demon\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3508, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\demon\AppData\Local\Temp\ipykernel_33004\1666654381.py", line 1, in <module>
    model.fit(
  File "c:\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
    raise e.with_traceback(filtered_tb) from None
  File "c:\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 67, in error_handler
    filtered_tb = _process_traceback_frames(e.__traceback__)
                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
TypeError: Model.fit() got an unexpected keyword argument 'nb_epoch'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\demon\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 2105, in showt

In [136]:
# Train on 1055 samples, validate on 118 samples

# Epoch 1/500
# 1055/1055 [==============================] - 6s - loss: 54.5698 - acc: 0.0000e+00 - val_loss: 56.6067 - val_acc: 0.0000e+00
# Epoch 2/500
# 1055/1055 [==============================] - 4s - loss: 54.2361 - acc: 0.0000e+00 - val_loss: 56.1267 - val_acc: 0.0000e+00
# Epoch 3/500
# 1055/1055 [==============================] - 3s - loss: 53.7573 - acc: 0.0000e+00 - val_loss: 55.5052 - val_acc: 0.0000e+00
# Epoch 4/500
# 1055/1055 [==============================] - 3s - loss: 53.1317 - acc: 0.0000e+00 - val_loss: 54.6624 - val_acc: 0.0000e+00
# Epoch 5/500
# 1055/1055 [==============================] - 3s - loss: 52.2918 - acc: 0.0000e+00 - val_loss: 53.5170 - val_acc: 0.0000e+00
# Epoch 6/500
# 1055/1055 [==============================] - 3s - loss: 51.1431 - acc: 0.0000e+00 - val_loss: 52.0444 - val_acc: 0.0000e+00
# Epoch 7/500
# 1055/1055 [==============================] - 3s - loss: 49.7124 - acc: 0.0000e+00 - val_loss: 50.2901 - val_acc: 0.0000e+00
# Epoch 8/500
# 1055/1055 [==============================] - 3s - loss: 48.0304 - acc: 0.0000e+00 - val_loss: 48.4176 - val_acc: 0.0000e+00
# Epoch 9/500
# 1055/1055 [==============================] - 3s - loss: 46.2073 - acc: 0.0000e+00 - val_loss: 46.4271 - val_acc: 0.0000e+00
# Epoch 10/500
# 1055/1055 [==============================] - 3s - loss: 44.2880 - acc: 0.0000e+00 - val_loss: 44.3134 - val_acc: 0.0000e+00

# Epoch 490/500
# 1055/1055 [==============================] - 6s - loss: 3.3352 - acc: 0.0000e+00 - val_loss: 0.2747 - val_acc: 0.0000e+00
# Epoch 491/500
# 1055/1055 [==============================] - 6s - loss: 3.3220 - acc: 0.0000e+00 - val_loss: 0.3347 - val_acc: 0.0000e+00
# Epoch 492/500
# 1055/1055 [==============================] - 6s - loss: 3.3430 - acc: 0.0000e+00 - val_loss: 0.3876 - val_acc: 0.0000e+00
# Epoch 493/500
# 1055/1055 [==============================] - 6s - loss: 3.3882 - acc: 0.0000e+00 - val_loss: 0.3919 - val_acc: 0.0000e+00
# Epoch 494/500
# 1055/1055 [==============================] - 6s - loss: 3.3117 - acc: 0.0000e+00 - val_loss: 0.3515 - val_acc: 0.0000e+00
# Epoch 495/500
# 1055/1055 [==============================] - 7s - loss: 3.4429 - acc: 0.0000e+00 - val_loss: 0.3073 - val_acc: 0.0000e+00
# Epoch 496/500
# 1055/1055 [==============================] - 7s - loss: 3.3748 - acc: 0.0000e+00 - val_loss: 0.2877 - val_acc: 0.0000e+00
# Epoch 497/500
# 1055/1055 [==============================] - 6s - loss: 3.3394 - acc: 0.0000e+00 - val_loss: 0.2870 - val_acc: 0.0000e+00
# Epoch 498/500
# 1055/1055 [==============================] - 6s - loss: 3.3265 - acc: 0.0000e+00 - val_loss: 0.2725 - val_acc: 0.0000e+00
# Epoch 499/500
# 1055/1055 [==============================] - 6s - loss: 3.3056 - acc: 0.0000e+00 - val_loss: 0.2654 - val_acc: 0.0000e+00
# Epoch 500/500
# 1055/1055 [==============================] - 6s - loss: 3.3673 - acc: 0.0000e+00 - val_loss: 0.2697 - val_acc: 0.0000e+00

In [137]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Open    2479 non-null   float64
 1   High    2479 non-null   float64
 2   Close   2479 non-null   float64
dtypes: float64(3)
memory usage: 58.2 KB


In [138]:
trainScore = model.evaluate(X_train, y_train, verbose=0)
print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))

testScore = model.evaluate(X_test, y_test, verbose=0)
print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))

Train Score: 1.65 MSE (1.28 RMSE)
Test Score: 9.55 MSE (3.09 RMSE)


Predictions vs Real results

In [139]:
import matplotlib.pyplot as plt2

plt2.plot(p,color='red', label='prediction')
plt2.plot(y_test,color='blue', label='y_test')
plt2.legend(loc='upper left')
plt2.show()

NameError: name 'p' is not defined